In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import warnings 
warnings.filterwarnings(action='ignore')


import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, LSTM, InputLayer
from sklearn.ensemble import VotingClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn import metrics 
from tensorflow import keras
import random  
from tensorflow.keras.callbacks import EarlyStopping

seed_num = 42

gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
    try:
        for i in range(len(gpus)):
            tf.config.experimental.set_memory_growth(gpus[i], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)

seed_num = 42
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4068).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

X_train.shape, y_train.shape, X_test.shape, y_test.shape


def get_model():
    lstm = Sequential()
    lstm.add(InputLayer(input_shape=(x.shape[1],x.shape[2])))
    lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(Dropout(0.2))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
    lstm.add(Dropout(0.2))
    lstm.add(Dense(units=1, activation='sigmoid'))

    lstm.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), 
                          loss = "binary_crossentropy", metrics=['acc'])
    return lstm

from keras.wrappers.scikit_learn import KerasClassifier

class MyKerasClassifier(KerasClassifier):
    def fit(self, x, y, sample_weight=None, **kwargs):
        y = np.array(y)
        if len(y.shape) == 2 and y.shape[1] > 1:
            self.classes_ = np.arange(y.shape[1])
        elif (len(y.shape) == 2 and y.shape[1] == 1) or len(y.shape) == 1:
            self.classes_ = np.unique(y)
            y = np.searchsorted(self.classes_, y)
        else:
            raise ValueError('Invalid shape for y: ' + str(y.shape))
        self.n_classes_ = len(self.classes_)
        #---------------수정---------------
        if sample_weight is not None:
            print('sample weight : ', sample_weight)
            if sample_weight[0] == 0.00016175994823681658:
                print('x, y', x.shape, x.sum().sum())
                return super(MyKerasClassifier, self).fit(x, y, **kwargs)
            weights = sample_weight / sum(sample_weight)
            random_range = [(sum(weights[:i]), sum(weights[:i])+weights[i]) if i!=0 else (0, weights[i]) for i in range(len(weights))]
            random_nums = [random.random() for _ in range(len(weights))]
            idx_list = []
            for i in random_nums:
                for j in random_range:
                    if j[0] < i <= j[1]:
                        idx_list.append(random_range.index(j))
                        break
            new_x = x[idx_list, :, :]
            new_y = y[idx_list]
            print(new_x.sum().sum())
            print('new_x, new_y', new_x.shape, new_y.shape)
            return super(MyKerasClassifier, self).fit(new_x, new_y, **kwargs)
        
    def predict(self, x, **kwargs):
        return super(MyKerasClassifier, self).predict(x)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


Using TensorFlow backend.


# Adaboost-LSTM

In [3]:
%%time
from tensorflow.keras.callbacks import ModelCheckpoint

with tf.device('/device:GPU:1'):
    early_stop = EarlyStopping(monitor='val_loss', patience=30, verbose=1, restore_best_weights=True)
#     cb_checkpoint = ModelCheckpoint(filepath='./models/adaboost_lstm1.h5', monitor='val_acc',
#                                     verbose=1, save_best_only=True)
#     base_estimator = MyKerasClassifier(build_fn=get_model, epochs=50, batch_size=128, validation_split=0.25, callbacks=[early_stop])
    base_estimator = MyKerasClassifier(build_fn=get_model, epochs=70, batch_size=4096, validation_split=0.25)

    boosted_classifier = AdaBoostClassifier(base_estimator=base_estimator, n_estimators=60, random_state=42, learning_rate=1)
    
    print("Adaboost LSTM Start")
    boosted_classifier.fit(X_train, y_train)
    preds = boosted_classifier.predict(X_test)

    precision = precision_score(y_test, preds)
    recall = recall_score(y_test, preds)
    f1 = f1_score(y_test, preds)
    roc_auc = roc_auc_score(y_test, preds)
    acc = accuracy_score(y_test, preds)

    print(f'Adaboost accuracy : {acc}, precision : {precision}, recall : {recall}, f1 : {f1}, roc_auc : {roc_auc}')

Adaboost LSTM Start
sample weight :  [0.00016176 0.00016176 0.00016176 ... 0.00016176 0.00016176 0.00016176]
x, y (6182, 10, 4068) 1644947.0
Epoch 1/70
2/2 [==============================] - 7s 779ms/step - loss: 0.6868 - acc: 0.5621 - val_loss: 0.6661 - val_acc: 0.6177
Epoch 2/70
2/2 [==============================] - 0s 159ms/step - loss: 0.6768 - acc: 0.5973 - val_loss: 0.6657 - val_acc: 0.6177
Epoch 3/70
2/2 [==============================] - 0s 162ms/step - loss: 0.6746 - acc: 0.6059 - val_loss: 0.6667 - val_acc: 0.6177
Epoch 4/70
2/2 [==============================] - 0s 162ms/step - loss: 0.6765 - acc: 0.6083 - val_loss: 0.6660 - val_acc: 0.6177
Epoch 5/70
2/2 [==============================] - 0s 182ms/step - loss: 0.6810 - acc: 0.6048 - val_loss: 0.6651 - val_acc: 0.6177
Epoch 6/70
2/2 [==============================] - 0s 167ms/step - loss: 0.6777 - acc: 0.6046 - val_loss: 0.6653 - val_acc: 0.6177
Epoch 7/70
2/2 [==============================] - 0s 161ms/step - loss: 0.6737 

2/2 [==============================] - 0s 137ms/step - loss: 0.3351 - acc: 0.8770 - val_loss: 0.5336 - val_acc: 0.7626
Epoch 58/70
2/2 [==============================] - 0s 133ms/step - loss: 0.3237 - acc: 0.8861 - val_loss: 0.5391 - val_acc: 0.7626
Epoch 59/70
2/2 [==============================] - 0s 141ms/step - loss: 0.3121 - acc: 0.8904 - val_loss: 0.5450 - val_acc: 0.7594
Epoch 60/70
2/2 [==============================] - 0s 131ms/step - loss: 0.3113 - acc: 0.8913 - val_loss: 0.5521 - val_acc: 0.7633
Epoch 61/70
2/2 [==============================] - 0s 136ms/step - loss: 0.3021 - acc: 0.8975 - val_loss: 0.5582 - val_acc: 0.7620
Epoch 62/70
2/2 [==============================] - 0s 140ms/step - loss: 0.2978 - acc: 0.9001 - val_loss: 0.5625 - val_acc: 0.7639
Epoch 63/70
2/2 [==============================] - 0s 140ms/step - loss: 0.2901 - acc: 0.8999 - val_loss: 0.5666 - val_acc: 0.7665
Epoch 64/70
2/2 [==============================] - 0s 144ms/step - loss: 0.2829 - acc: 0.9021 -

2/2 [==============================] - 0s 136ms/step - loss: 0.6690 - acc: 0.5981 - val_loss: 0.6648 - val_acc: 0.6300
Epoch 44/70
2/2 [==============================] - 0s 140ms/step - loss: 0.6629 - acc: 0.6130 - val_loss: 0.6579 - val_acc: 0.6688
Epoch 45/70
2/2 [==============================] - 0s 137ms/step - loss: 0.6540 - acc: 0.6521 - val_loss: 0.6481 - val_acc: 0.6662
Epoch 46/70
2/2 [==============================] - 0s 146ms/step - loss: 0.6436 - acc: 0.6739 - val_loss: 0.6396 - val_acc: 0.6772
Epoch 47/70
2/2 [==============================] - 0s 141ms/step - loss: 0.6275 - acc: 0.7062 - val_loss: 0.6251 - val_acc: 0.6934
Epoch 48/70
2/2 [==============================] - 0s 143ms/step - loss: 0.6116 - acc: 0.7187 - val_loss: 0.6102 - val_acc: 0.7050
Epoch 49/70
2/2 [==============================] - 0s 137ms/step - loss: 0.5904 - acc: 0.7396 - val_loss: 0.6169 - val_acc: 0.6643
Epoch 50/70
2/2 [==============================] - 0s 135ms/step - loss: 0.5803 - acc: 0.7299 -

2/2 [==============================] - 0s 136ms/step - loss: 0.6959 - acc: 0.4981 - val_loss: 0.6923 - val_acc: 0.5213
Epoch 30/70
2/2 [==============================] - 0s 143ms/step - loss: 0.6933 - acc: 0.5220 - val_loss: 0.6920 - val_acc: 0.5213
Epoch 31/70
2/2 [==============================] - 0s 136ms/step - loss: 0.6928 - acc: 0.5201 - val_loss: 0.6917 - val_acc: 0.5213
Epoch 32/70
2/2 [==============================] - 0s 141ms/step - loss: 0.6937 - acc: 0.5183 - val_loss: 0.6914 - val_acc: 0.5213
Epoch 33/70
2/2 [==============================] - 0s 140ms/step - loss: 0.6953 - acc: 0.5060 - val_loss: 0.6911 - val_acc: 0.5213
Epoch 34/70
2/2 [==============================] - 0s 141ms/step - loss: 0.6932 - acc: 0.5166 - val_loss: 0.6908 - val_acc: 0.5213
Epoch 35/70
2/2 [==============================] - 0s 147ms/step - loss: 0.6928 - acc: 0.5224 - val_loss: 0.6905 - val_acc: 0.5213
Epoch 36/70
2/2 [==============================] - 0s 137ms/step - loss: 0.6937 - acc: 0.5170 -

2/2 [==============================] - 0s 145ms/step - loss: 0.6953 - acc: 0.5101 - val_loss: 0.6930 - val_acc: 0.5071
Epoch 16/70
2/2 [==============================] - 0s 150ms/step - loss: 0.6964 - acc: 0.5093 - val_loss: 0.6930 - val_acc: 0.5071
Epoch 17/70
2/2 [==============================] - 0s 144ms/step - loss: 0.6984 - acc: 0.5017 - val_loss: 0.6931 - val_acc: 0.5071
Epoch 18/70
2/2 [==============================] - 0s 140ms/step - loss: 0.6972 - acc: 0.5056 - val_loss: 0.6932 - val_acc: 0.5071
Epoch 19/70
2/2 [==============================] - 0s 145ms/step - loss: 0.6971 - acc: 0.5080 - val_loss: 0.6933 - val_acc: 0.5071
Epoch 20/70
2/2 [==============================] - 0s 137ms/step - loss: 0.6966 - acc: 0.5006 - val_loss: 0.6934 - val_acc: 0.5071
Epoch 21/70
2/2 [==============================] - 0s 146ms/step - loss: 0.6969 - acc: 0.5056 - val_loss: 0.6935 - val_acc: 0.5071
Epoch 22/70
2/2 [==============================] - 0s 154ms/step - loss: 0.6982 - acc: 0.5002 -

...

2/2 [==============================] - 0s 172ms/step - loss: 0.6951 - acc: 0.4981 - val_loss: 0.6933 - val_acc: 0.4715
Epoch 20/70
2/2 [==============================] - 0s 154ms/step - loss: 0.6973 - acc: 0.4920 - val_loss: 0.6932 - val_acc: 0.4715
Epoch 21/70
2/2 [==============================] - 0s 157ms/step - loss: 0.6963 - acc: 0.4929 - val_loss: 0.6928 - val_acc: 0.5285
Epoch 22/70
2/2 [==============================] - 0s 136ms/step - loss: 0.6945 - acc: 0.5043 - val_loss: 0.6925 - val_acc: 0.5285
Epoch 23/70
2/2 [==============================] - 0s 144ms/step - loss: 0.6966 - acc: 0.4957 - val_loss: 0.6924 - val_acc: 0.5285
Epoch 24/70
2/2 [==============================] - 0s 148ms/step - loss: 0.6947 - acc: 0.5078 - val_loss: 0.6925 - val_acc: 0.5285
Epoch 25/70
2/2 [==============================] - 0s 146ms/step - loss: 0.6951 - acc: 0.5067 - val_loss: 0.6925 - val_acc: 0.5285
Epoch 26/70
2/2 [==============================] - 0s 137ms/step - loss: 0.6948 - acc: 0.5093 -

2/2 [==============================] - 0s 138ms/step - loss: 0.7003 - acc: 0.5101 - val_loss: 0.6930 - val_acc: 0.5155
Epoch 6/70
2/2 [==============================] - 0s 138ms/step - loss: 0.7001 - acc: 0.5032 - val_loss: 0.6927 - val_acc: 0.5155
Epoch 7/70
2/2 [==============================] - 0s 137ms/step - loss: 0.6995 - acc: 0.4948 - val_loss: 0.6929 - val_acc: 0.5155
Epoch 8/70
2/2 [==============================] - 0s 144ms/step - loss: 0.6986 - acc: 0.4965 - val_loss: 0.6937 - val_acc: 0.4845
Epoch 9/70
2/2 [==============================] - 0s 135ms/step - loss: 0.6989 - acc: 0.5056 - val_loss: 0.6947 - val_acc: 0.4845
Epoch 10/70
2/2 [==============================] - 0s 146ms/step - loss: 0.6995 - acc: 0.4974 - val_loss: 0.6953 - val_acc: 0.4845
Epoch 11/70
2/2 [==============================] - 0s 153ms/step - loss: 0.6992 - acc: 0.5019 - val_loss: 0.6955 - val_acc: 0.4845
Epoch 12/70
2/2 [==============================] - 0s 141ms/step - loss: 0.6967 - acc: 0.5039 - val

2/2 [==============================] - 0s 137ms/step - loss: 0.6949 - acc: 0.4989 - val_loss: 0.6928 - val_acc: 0.5155
Epoch 68/70
2/2 [==============================] - 0s 140ms/step - loss: 0.6945 - acc: 0.4987 - val_loss: 0.6928 - val_acc: 0.5155
Epoch 69/70
2/2 [==============================] - 0s 141ms/step - loss: 0.6938 - acc: 0.4996 - val_loss: 0.6929 - val_acc: 0.5155
Epoch 70/70
2/2 [==============================] - 0s 139ms/step - loss: 0.6942 - acc: 0.4950 - val_loss: 0.6929 - val_acc: 0.5155
sample weight :  [4.57816835e-04 1.85308373e-05 2.83643489e-05 ... 2.05333606e-05
 1.65788927e-04 4.86534078e-05]
1319798.0
new_x, new_y (6182, 10, 4068) (6182,)
Epoch 1/70
2/2 [==============================] - 6s 564ms/step - loss: 0.7005 - acc: 0.5026 - val_loss: 0.6932 - val_acc: 0.5058
Epoch 2/70
2/2 [==============================] - 0s 147ms/step - loss: 0.6971 - acc: 0.5164 - val_loss: 0.6933 - val_acc: 0.5058
Epoch 3/70
2/2 [==============================] - 0s 144ms/step - 

2/2 [==============================] - 0s 148ms/step - loss: 0.6927 - acc: 0.5073 - val_loss: 0.6933 - val_acc: 0.4942
Epoch 54/70
2/2 [==============================] - 0s 141ms/step - loss: 0.6941 - acc: 0.5032 - val_loss: 0.6935 - val_acc: 0.4942
Epoch 55/70
2/2 [==============================] - 0s 140ms/step - loss: 0.6935 - acc: 0.5035 - val_loss: 0.6936 - val_acc: 0.4942
Epoch 56/70
2/2 [==============================] - 0s 145ms/step - loss: 0.6958 - acc: 0.4881 - val_loss: 0.6938 - val_acc: 0.4942
Epoch 57/70
2/2 [==============================] - 0s 137ms/step - loss: 0.6947 - acc: 0.5011 - val_loss: 0.6939 - val_acc: 0.4942
Epoch 58/70
2/2 [==============================] - 0s 133ms/step - loss: 0.6942 - acc: 0.5017 - val_loss: 0.6939 - val_acc: 0.4942
Epoch 59/70
2/2 [==============================] - 0s 135ms/step - loss: 0.6933 - acc: 0.5047 - val_loss: 0.6938 - val_acc: 0.4942
Epoch 60/70
2/2 [==============================] - 0s 146ms/step - loss: 0.6939 - acc: 0.5101 -

2/2 [==============================] - 0s 147ms/step - loss: 0.6938 - acc: 0.5078 - val_loss: 0.6920 - val_acc: 0.5239
Epoch 40/70
2/2 [==============================] - 0s 139ms/step - loss: 0.6934 - acc: 0.5091 - val_loss: 0.6920 - val_acc: 0.5239
Epoch 41/70
2/2 [==============================] - 0s 132ms/step - loss: 0.6938 - acc: 0.5173 - val_loss: 0.6921 - val_acc: 0.5239
Epoch 42/70
2/2 [==============================] - 0s 132ms/step - loss: 0.6929 - acc: 0.5272 - val_loss: 0.6921 - val_acc: 0.5239
Epoch 43/70
2/2 [==============================] - 0s 149ms/step - loss: 0.6936 - acc: 0.5162 - val_loss: 0.6921 - val_acc: 0.5239
Epoch 44/70
2/2 [==============================] - 0s 142ms/step - loss: 0.6940 - acc: 0.5147 - val_loss: 0.6920 - val_acc: 0.5239
Epoch 45/70
2/2 [==============================] - 0s 145ms/step - loss: 0.6927 - acc: 0.5188 - val_loss: 0.6920 - val_acc: 0.5239
Epoch 46/70
2/2 [==============================] - 0s 135ms/step - loss: 0.6948 - acc: 0.5166 -

In [ ]:
Adaboost accuracy : 0.7320388349514563, precision : 0.7601214574898786, recall : 0.8092672413793104, f1 : 0.7839248434237996, roc_auc : 0.7125752738501091

In [3]:
y_pred_test = reload_model.predict(X_test)

y_pred_test[y_pred_test>0.5]=1
y_pred_test[y_pred_test<=0.5]=0
precision = precision_score(y_test, y_pred_test)
recall = recall_score(y_test, y_pred_test)
f1 = f1_score(y_test, y_pred_test)
roc_auc = roc_auc_score(y_test, y_pred_test)
acc = accuracy_score(y_test, y_pred_test)

print(f'accuracy : {acc}, precision : {precision}, recall : {recall}, f1 : {f1}, roc_auc : {roc_auc}')

NameError: name 'reload_model' is not defined

## 모델저장

In [8]:
import pickle
import joblib

joblib.dump(boosted_classifier, './models/adaboost_lstm.pkl') 
reload_model = joblib.load('./models/adaboost_lstm.pkl') 

INFO:tensorflow:Assets written to: ram://4fc3a096-2c5e-42aa-8ed7-0d35dd718c6c/assets


INFO:tensorflow:Assets written to: ram://3f4d161d-ae84-45e1-b949-b9e25640463a/assets


INFO:tensorflow:Assets written to: ram://3f4d161d-ae84-45e1-b949-b9e25640463a/assets


INFO:tensorflow:Assets written to: ram://aea38500-abca-43c1-9f03-3d652beb35d0/assets


INFO:tensorflow:Assets written to: ram://aea38500-abca-43c1-9f03-3d652beb35d0/assets


...

INFO:tensorflow:Assets written to: ram://94288a7f-24f6-4f72-9a74-76d071b01a68/assets


INFO:tensorflow:Assets written to: ram://53037386-12b4-4385-b5cb-98f69aed3181/assets


INFO:tensorflow:Assets written to: ram://53037386-12b4-4385-b5cb-98f69aed3181/assets


INFO:tensorflow:Assets written to: ram://56c8a39b-d555-4124-a103-af0a24f06b89/assets


INFO:tensorflow:Assets written to: ram://56c8a39b-d555-4124-a103-af0a24f06b89/assets


## etc..

In [10]:
a = [i for i in boosted_classifier.staged_predict_proba(X_test)]
len(a[0])
a

[array([[0.21709043, 0.7829096 ],
        [0.1775779 , 0.8224221 ],
        [0.12006653, 0.8799335 ],
        ...,
        [0.18669136, 0.81330866],
        [0.10798669, 0.8920133 ],
        [0.28414887, 0.7158511 ]], dtype=float32),
 array([[0.3096022 , 0.6903978 ],
        [0.24463809, 0.7553619 ],
        [0.12932858, 0.8706714 ],
        ...,
        [0.41682047, 0.58317953],
        [0.08030998, 0.9196901 ],
        [0.14576423, 0.85423577]], dtype=float32),
 array([[0.49377504, 0.50622493],
        [0.58474076, 0.41525927],
        [0.33875763, 0.66124237],
        ...,
        [0.45829618, 0.54170376],
        [0.06277613, 0.93722385],
        [0.28231055, 0.7176894 ]], dtype=float32),
 array([[0.6876445 , 0.31235552],
        [0.3763796 , 0.6236204 ],
        [0.208996  , 0.791004  ],
        ...,
        [0.28230792, 0.71769214],
        [0.23227866, 0.76772135],
        [0.30269474, 0.69730526]], dtype=float32),
 array([[0.49829653, 0.50170344],
        [0.26000848, 0.7399915

In [11]:
boosted_classifier.estimator_weights_

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1.])

In [12]:
boosted_classifier.estimator_errors_

array([0.1821417 , 0.17475991, 0.14589192, 0.15551748, 0.13755002,
       0.17682738, 0.22586276, 0.1431941 , 0.12552032, 0.17962682,
       0.18907787, 0.15463544, 0.1608663 , 0.24392606, 0.10908219,
       0.06825134, 0.13479391, 0.15155727, 0.12661465, 0.12899345])

In [100]:
y_pred_test

array([0, 0, 1, ..., 1, 1, 1])

In [101]:
boosted_classifier.estimators_

In [102]:
boosted_classifier.estimator_weights_

array([1., 1., 1., 1., 1.])

In [103]:
boosted_classifier.estimator_errors_

array([0.19136202, 0.19596451, 0.16152457, 0.14712343, 0.16450197])